In [116]:
import kagglehub
import pandas as pd
import os
import re
import numpy as np

In [ ]:
# Download latest version
path = kagglehub.dataset_download("anushabellam/amazon-reviews-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/anushabellam/amazon-reviews-dataset/versions/1


In [ ]:
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        csv_file_path = os.path.join(path, filename)


df = pd.read_csv(csv_file_path)
df['review_body']=df['review_body'].str.lower()
df['actual'] = df['Sentiment_pc'].apply(lambda x: 1 if x == 'positive' else 0)
df

,Unnamed: 0,market_place,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Sentiment_pc,actual
0,0,"""US""","""22873041""","""R3ARRMDEGED8RD""","""B00KJWQIIC""","""335625766""","""Plemo 14-Inch Laptop Sleeve Case Waterproof F...","""PC""",1,0,0,0 \t(N),1 \t(Y),"""Pleasantly surprised""","""i was very surprised at the high quality of t...",2015-08-31,positive,1
1,1,"""US""","""30088427""","""RQ28TSA020Y6J""","""B013ALA9LA""","""671157305""","""TP-Link OnHub AC1900 Wireless Wi-Fi Router""","""PC""",1,24,31,0 \t(N),0 \t(N),"""OnHub is a pretty no nonsense type router tha...","""i am a google employee and had to chance to u...",2015-08-31,positive,1
2,2,"""US""","""20329786""","""RUXJRZCT6953M""","""B00PML2GQ8""","""982036237""","""AmazonBasics USB 3.0 A Male to A Male Cable -...","""PC""",0,2,2,0 \t(N),0 \t(N),"""None of them worked. No functionality at all.""","""bought cables in 3ft, 6ft and 9ft. none of th...",2015-08-31,negaitve,0
3,3,"""US""","""14215710""","""R7EO0UO6BPB71""","""B001NS0OZ4""","""576587596""","""Transcend P8 15-in-1 USB 2.0 Flash Memory Car...","""PC""",0,0,0,0 \t(N),1 \t(Y),"""just keep searching.""","""nope, cheap and slow""",2015-08-31,negaitve,0
4,4,"""US""","""38264512""","""R39NJY2YJ1JFSV""","""B00AQMTND2""","""964759214""","""Aleratec SATA Data Cable 2.0 20in Serial ATA ...","""PC""",1,0,0,0 \t(N),1 \t(Y),"""Five Stars""","""excellent! great value and does the job.""",2015-08-31,positive,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,"""US""","""15553382""","""R12HS6NXPUZMJR""","""B001AR6MI6""","""31174579""","""StarTech 1ft Micro USB Cable""","""PC""",0,3,4,0 \t(N),1 \t(Y),"""Nice cable, but wrong style""","""i ordered the right angle cable, but received...",2015-08-31,negaitve,0
96,96,"""US""","""466731""","""R2I3UB5QK1RDIR""","""B005O6Z028""","""838770124""","""Thrustmaster Ferrari 458 Italia Gaming Steeri...","""PC""",1,0,0,0 \t(N),1 \t(Y),"""Five Stars""","""works greater""",2015-08-31,positive,1
97,97,"""US""","""24439106""","""R2SB4XQBVNYZ0M""","""B00HK5IQ6S""","""512172051""","""Genius Professional Colorful LED Backlit USB ...","""PC""",1,0,0,0 \t(N),1 \t(Y),"""Seems like a decent keyboard""","""i had to return this (for a reason i can't re...",2015-08-31,positive,1
98,98,"""US""","""11490310""","""R2YB7JO9L5H8AP""","""B00SGNZM88""","""867561596""","""Demupai 5.9 Feet MHL Micro USB to HDMI 1080P ...","""PC""",1,2,2,0 \t(N),1 \t(Y),"""Disappointment""","""cords don't last very long. disappointment""",2015-08-31,positive,1


In [115]:
def tokenize(text):
  """
    gets a string and turn it into token by the following rules:
  -treat . ! ? , " ; (period, exclamation, question mark, comma, double quote, and semicolon) as tokens. All other punctuation can be ignored, not extracted as a token or part of a token except where noted below.
  -allow contractions and possesives where ' appears in the middle of the word and ≤ 3 letters appear after the apostrophe ex. can't, you're, i'm, Marvin's, would've
  -allows tokens to consist of both letters and integers e.g. "24k" "32in"
  -allow - (dash) to appear in the middle of a token
  Args:
    param text :string

  Returns:
    tokens: list of tokens
  """
  pattern = r"\b\w+(?:[-']\w+)?\b|[.,!?\";]"

  tokens = re.findall(pattern, text)

  return tokens

In [ ]:
df['tokens'] = df['review_body'].apply(tokenize)
tokens = df['review_body'].apply(tokenize)

In [ ]:
def lexicaScore(lexica, tokens,trash):
  """
  gets a dict tokens and trash holder and return prediction for each token
  Args:
    param lexica : dictonary containing positive and negetive words
    param tokens:  list of list of tokens
    param trash: float number betweeen 1 and 0

  Returns:
    predict_score: list of 1 and 0
  """

  predict_score = []
  for token in tokens:
    positive_score = 0
    negative_score = 0
    for word in token:
      if word in lexica["Positive"]:
        positive_score += 1
      elif word in lexica["Negetive"]:
        negative_score += 1
    positive_score = positive_score/len(token)
    negative_score = negative_score/len(token)
    if positive_score-negative_score >trash:
      predict_score.append(1)
    else:
      predict_score.append(0)
  return predict_score


In [ ]:
def calculate_accuracy(df):
  """
  gets a df and return accuracy
  Args:
    param df: pandas df containing actual and predict columns
  Returns:
    accuracy: float number
  """
  correct_predictions = (df['actual'] == df['predict']).sum()
  total_predictions = len(df)
  accuracy = correct_predictions / total_predictions
  return accuracy

In [ ]:
lexica = {"Positive":[
        'good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic',
        'awesome', 'love', 'best', 'perfect', 'brilliant', 'outstanding',
        'superb', 'delightful', 'enjoyable', 'pleasant', 'pleased', 'happy',
        'recommend', 'impressive', 'helpful', 'interesting', 'nice', 'well'
    ],"Negetive":[
        'bad', 'worst', 'terrible', 'horrible', 'awful', 'disappointing',
        'poor', 'useless', 'waste', 'hate', 'problem', 'issues', 'terrible',
        'boring', 'disappointed', 'frustrating', 'annoying', 'unhappy',
        'difficulty', 'difficult', 'worse', 'terrible', 'nightmare'
    ]}

In [ ]:
df['predict'] =lexicaScore(lexica,df["tokens"],trash=0.05)
df[['predict','actual']]
calculate_accuracy(df)

0.6


In [ ]:
trash_list = [0.2, 0.1, 0.05,0.01,0.005]
for trash in trash_list:
    df['predict'] = lexicaScore(lexica, df["tokens"], trash=trash)
    accuracy = calculate_accuracy(df)
    print(f'Trash = {trash}, Accuracy = {accuracy}')
  #we can see that for Trash 0.01 and 0.005 the accuracy is 0.75

Trash = 0.2, Accuracy = 0.37
Trash = 0.1, Accuracy = 0.46
Trash = 0.05, Accuracy = 0.6
Trash = 0.01, Accuracy = 0.75
Trash = 0.005, Accuracy = 0.75


In [ ]:
def create_vocab(tokens,min_size=5):
  """
  looking at all words we have and creating a vocabullary with words that appear more than min_size times
  Args:
    param tokens : list of tokens
    min_size : integer

  Return:
    a dictonary
  """
  all_tokens =[]
  vocab = {}
  for token in tokens:
    all_tokens.extend(token)
  for word in all_tokens:
    if all_tokens.count(word) > 5:
      vocab[word] = all_tokens.count(word)
  return vocab

In [ ]:
vocab = create_vocab(tokens,5)
len(vocab)

173

In [ ]:
def extractMultiHot(tokens,vocab):
  """
  Doing one hot encoding for each token
  Args:
    param tokens : list of tokens
    vocab: a dictonary

  return:
    a list of vectors
  """
  multi_hot = []
  for token in tokens:
    vector = []
    for word in vocab:
      if word in token:
        vector.append(1)
      else:
        vector.append(0)
    multi_hot.append(vector)
  return multi_hot

In [ ]:
df['OneHot'] = extractMultiHot(tokens,vocab)

,OneHot
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
2,"[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
4,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
...,...
95,"[1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, ..."
96,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,"[1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, ..."
98,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [121]:
def normalizedLogLoss(ypred, ytrue):
  """
  Calculates the normalized log loss for given list of prediction probabilities and true values
  Args:
    ypred: List of prediction probabilities
    ytrue: List of true values

  Returns:
    log_likelihood: float (Normalized log loss)
  """




  epsilon = 1e-15  #small value to aviod log(0)
  ypred = np.clip(ypred, epsilon, 1 - epsilon)

  ytrue = np.array(ytrue)
  ypred = np.array(ypred)

  log_likelihood = -np.mean(ytrue * np.log(ypred) + (1 - ytrue) * np.log(1 - ypred))

  return log_likelihood

0.22708064055624455
